In [1]:
import math as m
import numpy as np
import cv2
import os
import imageio
from scipy.optimize import fsolve
import matplotlib.pyplot as pl
import time
import sympy as sy

# Adatok kód: 87

In [2]:
omega = 8
h = 0.05
e = 0.03
l3 = 0.1
l4 = 0.12
s = 0.8*l3
periodtime = 2*m.pi/omega
l1 = 26.15067129/1000
l2 = 134.66748367/1000
dAB = 93.91053598/1000
xD = 0
yD = 0
yA = 0
xA = -dAB
yH = h
step = 360
fics = 80

Az bal oldali szélső értékhez tartozó Fi1 szög
A jobb oldali szélsőértékhez tartozó Fi1 szög
A Fi* -hoz tartozó szög
Solidworksből kiolvasot szögek segítségével

In [3]:
fibal = 58.679/180*m.pi
fijobb = (58.679 + 156.52)/180*m.pi
ficsillag = (180+fics)/180*m.pi
nbal = round(step*fibal/omega)
njobb = round(step*fijobb/omega)
ncsillag = round(step*ficsillag/omega)
tbal = fibal/omega
tjobb = fijobb/omega
tcsillag = ficsillag/omega
fikrit = [fibal,fijobb,ficsillag]
tkrit = [tbal, tjobb, tcsillag]

# Pályák

In [4]:
def f(x):
    fi1, fi2, fi3, xB, yB = x
    Fvector = np.array([
        xB - xA + l1*np.cos(fi1),
        yB - yA + l1*np.sin(fi1),
        xB + l2*np.cos(fi2) - (xD + l3*np.cos(fi3)),
        yB + l2*np.sin(fi2) - (yD + l3*np.sin(fi3)),
        fi1 - omega*t])
    return Fvector

t = 0
results = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
tspace = []
for i in range(round(periodtime*step)+1):
    tspace.append(t)
    if i != 0:
        solution = fsolve(f, solution)
    else:
        solution = fsolve(f, [0, 0.733, 1.187, -0.067, 0])
    currentFi3 = solution[2]
    xC = xD + l3*np.cos(currentFi3)
    yC = yD + l3*np.sin(currentFi3)
    xE = xD + s*np.cos(currentFi3)
    yE = yD + s*np.sin(currentFi3)
    Fi4 = np.arcsin((yH-yE)/l4)
    xH = xE + l4*np.cos(Fi4)
    results[0].append(solution[0])
    results[1].append(solution[1])
    results[2].append(solution[2])
    results[3].append(Fi4)
    results[4].append(xA)
    results[5].append(yA)
    results[6].append(solution[3])
    results[7].append(solution[4])
    results[8].append(xC)
    results[9].append(yC)
    results[10].append(xD)
    results[11].append(yD)
    results[12].append(xE)
    results[13].append(yE)
    results[14].append(xH)
    results[15].append(yH)
    t += 1/step

# Jakobi mátrix készítés

In [5]:
sfi1, sfi2, sfi3, sfi4, sxB, syB, sxE, syE, sxH=sy.symbols('sfi1, sfi2, sfi3, sfi4, sxB, syB, sxE, syE, sxH')
X = sy.Matrix([
    sxB - xA + l1*sy.cos(sfi1),
    syB - yA + l1*sy.sin(sfi1),
    sxB + l2*sy.cos(sfi2) - (xD + l3*sy.cos(sfi3)),
    syB + l2*sy.sin(sfi2) - (yD + l3*sy.sin(sfi3)),
    sxE - xD + s*sy.cos(sfi3),
    syE - yD + s*sy.sin(sfi3),
    sfi4 - sy.asin((yH-syE)/l4),
    sxH - sxE + l4*sy.cos(sfi4),
    sfi1 - omega*t
])
Q = sy.Matrix([sfi1, sfi2, sfi3, sfi4, sxB, syB, sxE, syE, sxH])
X.jacobian(Q)

Matrix([
[-0.02615067129*sin(sfi1),                        0,               0,               0, 1, 0,  0,                                                                        0, 0],
[ 0.02615067129*cos(sfi1),                        0,               0,               0, 0, 1,  0,                                                                        0, 0],
[                       0, -0.13466748367*sin(sfi2),   0.1*sin(sfi3),               0, 1, 0,  0,                                                                        0, 0],
[                       0,  0.13466748367*cos(sfi2),  -0.1*cos(sfi3),               0, 0, 1,  0,                                                                        0, 0],
[                       0,                        0, -0.08*sin(sfi3),               0, 0, 0,  1,                                                                        0, 0],
[                       0,                        0,  0.08*cos(sfi3),               0, 0, 0,  0,                    

# Numerikus kiértékelése a sebességvektornak (Jacobival)

In [6]:
J = X.jacobian(Q)
nJ = sy.lambdify((sfi1, sfi2, sfi3, sfi4, syE),J,"numpy")
JSpace = []
invJSpace = []
b = np.array([0,0,0,0,0,0,0,0,-omega])
v = []
a = []
for i in range(len(results[0])):
    fi1 = results[0][i]
    fi2 = results[1][i]
    fi3 = results[2][i]
    fi4 = results[3][i]
    yE = results[13][i]
    currentJ = nJ(fi1, fi2, fi3, fi4, yE)
    invCurrentJ = np.linalg.inv(currentJ)
    JSpace.append(currentJ)
    invJSpace.append(invCurrentJ)
    '''
    A sebességvektor elemei:
    sfi1, sfi2, sfi3, sfi4, sxB, syB, sxE, syE, sxH
    '''

    cv = np.dot(invCurrentJ,-b)
    try:
        ca = (v[i-1]-cv)*step
        a.append(ca)
    except:
        pass
    v.append(np.dot(invCurrentJ,-b))
a.append(ca)
a = np.array(a)
vmod = np.transpose(v)

# Fi3 az idő és Fi1 függvényében

In [0]:
plotFi1var = results[0]
plotFi3var = results[2]
plotFi3vvar = vmod[2]
plotFi3avar = a[...,2]
plotxHvar = results[14]
plotxHvvar = vmod[8]
plotxHavar = a[...,8]
plotFi3 = np.array([results[2] for i in range(3)]).ravel()
plotFi1 = np.array([results[0] for i in range(3)]).ravel()
plotFi3v = np.array([vmod[2] for i in range(3)]).ravel()
plotFi3a = np.array([a[...,2] for i in range(3)]).ravel()
plotxH = np.array([results[14] for i in range(3)]).ravel()
plotxHv = np.array([vmod[8] for i in range(3)]).ravel()
plotxHa = np.array([a[...,8] for i in range(3)]).ravel()
test = np.zeros(len(plotxH)*5)
test[:len(plotxH)] = plotxH
test[len(plotxH):len(plotxH)*2] = plotxH
test[len(plotxH)*2:len(plotxH)*3] = plotxH
test[len(plotxH)*3:len(plotxH)*4] = plotxH
test[len(plotxH)*4:len(plotxH)*5] = plotxH
testspace = [i/step for i in range(len(plotFi3))]
# t-fi3
pl.plot(testspace,plotFi3)
pl.scatter(tkrit[0],[plotFi3[nbal]], s=50, label='bal szélsőhelyzet')
pl.scatter(tkrit[1],[plotFi3[njobb]], s=50, label='jobb szélsőhelyzet')
pl.scatter(tkrit[2],[plotFi3[ncsillag]], s=50, label='Fi* helyzet')
pl.legend(loc='upper right')
pl.ylabel('Szöghelyzet [rad]')
pl.xlabel('Idő [s]')
pl.title('A hármas kar szöghelyzete az idő függvényében')
pl.show()
# t-fi3v
pl.plot(testspace,plotFi3v)
pl.scatter(tkrit[0],[plotFi3v[nbal]], s=50, label='bal szélsőhelyzet')
pl.scatter(tkrit[1],[plotFi3v[njobb]], s=50, label='jobb szélsőhelyzet')
pl.scatter(tkrit[2],[plotFi3v[ncsillag]], s=50, label='Fi* helyzet')
pl.legend(loc='upper right')
pl.ylabel('Szögsebesség [rad/s]')
pl.xlabel('Idő [s]')
pl.title('A hármas kar szögsebessége az idő függvényében')
pl.show()
# t-fi3a
pl.plot(testspace,plotFi3a)
pl.scatter(tkrit[0],[plotFi3a[nbal]], s=50, label='bal szélsőhelyzet')
pl.scatter(tkrit[1],[plotFi3a[njobb]], s=50, label='jobb szélsőhelyzet')
pl.scatter(tkrit[2],[plotFi3a[ncsillag]], s=50, label='Fi* helyzet')
pl.legend(loc='upper right')
pl.ylabel('Szöggyorsulás [rad/s^2]')
pl.xlabel('Idő [s]')
pl.title('A hármas kar szöggyorsulása az idő függvényében')
pl.show()
# fi1-fi3
pl.plot(plotFi1var,plotFi3var)
pl.scatter(fikrit[0],[plotFi3[nbal]], s=50, label='bal szélsőhelyzet')
pl.scatter(fikrit[1],[plotFi3[njobb]], s=50, label='jobb szélsőhelyzet')
pl.scatter(fikrit[2],[plotFi3[ncsillag]], s=50, label='Fi* helyzet')
pl.legend(loc='upper right')
pl.xlabel('Fi1 szöghelyzet [rad]')
pl.ylabel('Fi3 szöghelyzet [rad]')
pl.title('A hármas kar szöghelyzete\naz egyes kar szöghelyzetének függvényében')
pl.show()
# fi1-fi3v
pl.plot(plotFi1var,plotFi3vvar)
pl.scatter(fikrit[0],[plotFi3v[nbal]], s=50, label='bal szélsőhelyzet')
pl.scatter(fikrit[1],[plotFi3v[njobb]], s=50, label='jobb szélsőhelyzet')
pl.scatter(fikrit[2],[plotFi3v[ncsillag]], s=50, label='Fi* helyzet')
pl.legend(loc='lower right')
pl.xlabel('Fi1 szöghelyzet [rad]')
pl.ylabel('Fi3 szögsebesség [rad/s]')
pl.title('A hármas kar szögsebessége\naz egyes kar szöghelyzetének függvényében')
pl.show()
# fi1-fi3a
pl.plot(plotFi1var,plotFi3avar)
pl.scatter(fikrit[0],[plotFi3a[nbal]], s=50, label='bal szélsőhelyzet')
pl.scatter(fikrit[1],[plotFi3a[njobb]], s=50, label='jobb szélsőhelyzet')
pl.scatter(fikrit[2],[plotFi3a[ncsillag]], s=50, label='Fi* helyzet')
pl.legend(loc='upper right')
pl.xlabel('Fi1 szöghelyzet [rad]')
pl.ylabel('Fi3 szöggyorsulás [rad/s^2]')
pl.title('A hármas kar szöggyorsulása\naz egyes kar szöghelyzetének függvényében')
pl.show()
print(plotFi3[nbal],plotFi3[njobb],plotFi3[ncsillag])
print(plotFi3v[nbal],plotFi3v[njobb],plotFi3v[ncsillag])
print(plotFi3a[nbal],plotFi3a[njobb],plotFi3a[ncsillag])

# A H csukló az idő és f1 függvényében

In [7]:

# t-xH
pl.plot(testspace,plotxH)
pl.scatter(tkrit[0],[plotxH[nbal]], s=50, label='bal szélsőhelyzet')
pl.scatter(tkrit[1],[plotxH[njobb]], s=50, label='jobb szélsőhelyzet')
pl.scatter(tkrit[2],[plotxH[ncsillag]], s=50, label='Fi* helyzet')
pl.legend(loc='upper right')
pl.xlabel('Idő [s]')
pl.ylabel('Hx [m]')
pl.title('A H csukló x pozíciója az idő függvényében')
pl.show()
# t-xHv
pl.plot(testspace,plotxHv)
pl.scatter(tkrit[0],[plotxHv[nbal]], s=50, label='bal szélsőhelyzet')
pl.scatter(tkrit[1],[plotxHv[njobb]], s=50, label='jobb szélsőhelyzet')
pl.scatter(tkrit[2],[plotxHv[ncsillag]], s=50, label='Fi* helyzet')
pl.legend(loc='upper right')
pl.ylabel('Sebesség [m/s]')
pl.xlabel('Idő [s]')
pl.title('A H csukló sebessége az idő függvényében')
pl.show()
# t-xHa
pl.plot(testspace,plotxHa)
pl.scatter(tkrit[0],[plotxHa[nbal]], s=50, label='bal szélsőhelyzet')
pl.scatter(tkrit[1],[plotxHa[njobb]], s=50, label='jobb szélsőhelyzet')
pl.scatter(tkrit[2],[plotxHa[ncsillag]], s=50, label='Fi* helyzet')
pl.legend(loc='upper right')
pl.ylabel('Gyorsulás [m/s^2]')
pl.xlabel('Idő [s]')
pl.title('A H csukló gyorsulása az idő függvényében')
pl.show()
# t-xH

pl.plot(plotFi1var,plotxHvar)
pl.scatter(fikrit[0],[plotxH[nbal]], s=50, label='bal szélsőhelyzet')
pl.scatter(fikrit[1],[plotxH[njobb]], s=50, label='jobb szélsőhelyzet')
pl.scatter(fikrit[2],[plotxH[ncsillag]], s=50, label='Fi* helyzet')
pl.legend(loc='lower right')
pl.xlabel('Fi1 [rad]')
pl.ylabel('Hx [m]')
pl.title('A H csukló x pozíciója Fi1 függvényében')
pl.show()
# t-xHv
pl.plot(plotFi1var,plotxHvvar)
pl.scatter(fikrit[0],[plotxHv[nbal]], s=50, label='bal szélsőhelyzet')
pl.scatter(fikrit[1],[plotxHv[njobb]], s=50, label='jobb szélsőhelyzet')
pl.scatter(fikrit[2],[plotxHv[ncsillag]], s=50, label='Fi* helyzet')
pl.legend(loc='lower right')
pl.ylabel('Sebesség [m/s]')
pl.xlabel('Fi1 [rad]')
pl.title('A H csukló sebessége Fi1 függvényében')
pl.show()
# t-xHa
pl.plot(plotFi1var,plotxHavar)
pl.scatter(fikrit[0],[plotxHa[nbal]], s=50, label='bal szélsőhelyzet')
pl.scatter(fikrit[1],[plotxHa[njobb]], s=50, label='jobb szélsőhelyzet')
pl.scatter(fikrit[2],[plotxHa[ncsillag]], s=50, label='Fi* helyzet')
pl.legend(loc='upper right')
pl.ylabel('Gyorsulás [m/s^2]')
pl.xlabel('Fi1 [rad]')
pl.title('A H csukló gyorsulása Fi1 függvényében')
pl.show()
print(plotxH[nbal],plotxH[njobb],plotxH[ncsillag])
print(plotxHv[nbal],plotxHv[njobb],plotxHv[ncsillag])
print(plotxHa[nbal],plotxHa[njobb],plotxHa[ncsillag])

NameError: name 'testspace' is not defined

# Bemenő nyomaték, kivett erő arány számítása

In [10]:
# Adott: xA, yA, xD, yD, yH
xB, yB, xC, yC, xE, yE, xH=sy.symbols('xB, yB, xC, yC, xE, yE, xH')
F = sy.Matrix([
    (yA-yB)**2 + (xA-xB)**2 - l1**2,
    (yC-yB)**2 + (xC-xB)**2 - l2**2,
    (yD-yC)**2 + (xD-xC)**2 - l3**2,
    (yD-yE)**2 + (xD-xE)**2 - s**2,
    (yC-yE)**2 + (xC-xE)**2 - (l3-s)**2,
    (yE-yH)**2 + (xE-xH)**2 - l4**2,
    xH
])
szabadsagfok = sy.Matrix([xB, yB, xC, yC, xE, yE, xH])
F.jacobian(szabadsagfok)

Matrix([
[2*xB + 0.18782107196,        2*yB,            0,            0,            0,            0,            0],
[         2*xB - 2*xC, 2*yB - 2*yC, -2*xB + 2*xC, -2*yB + 2*yC,            0,            0,            0],
[                   0,           0,         2*xC,         2*yC,            0,            0,            0],
[                   0,           0,            0,            0,         2*xE,         2*yE,            0],
[                   0,           0,  2*xC - 2*xE,  2*yC - 2*yE, -2*xC + 2*xE, -2*yC + 2*yE,            0],
[                   0,           0,            0,            0,  2*xE - 2*xH,   2*yE - 0.1, -2*xE + 2*xH],
[                   0,           0,            0,            0,            0,            0,            1]])

In [19]:
q = np.array(results.copy())
q = q[...,ncsillag]
'''
A fentebb felírt mátrixban szereplő szabadságfokok + vezetőkényszer vektora:
q = xB, yB, xC, yC, xE, yE, xH
Ahol xH a vezető tag aminek az előírt pályán kell mozognia
xB és yB helyen hat erő, mely a behajtó nyomatékból származik, és nagysága egyenlő Q-ban szereplő értékkel
'''
Jq = F.jacobian(szabadsagfok)
Jq = sy.lambdify([xB, yB, xC, yC, xE, yE, xH],Jq,"numpy")
nJq = Jq(q[6],q[7],q[8],q[9],q[12],q[13],q[14])
Jqt = np.transpose(nJq)
invJqt = np.linalg.inv(Jqt)
Q = np.array([10*(q[5]-q[7])/l1**2,10*(q[4]-q[6])/l1**2,0,0,0,0,0])
Q2 = np.array([10*np.sin(q[0])/l1,10*np.cos(q[0])/l1,0,0,0,0,0])
Q3 = np.array([-10*(q[7]-q[5])/l1**2,10*(q[6]-q[4])/l1**2,0,0,0,0,0])

vxh = vmod[8,ncsillag]
omega1 = vmod[0,ncsillag]
print(10*omega1/vxh)
print(vxh, omega1)

# lam1, lam2, lam3, lam4, lam5, lam6, lam7, lam8, nu

print(Q.dot(invJqt))
print(invJqt.dot(Q2))
print(invJqt.dot(Q3))

488.79825508072486
0.16366670537068106 8.0
[ 4.24368713e+03 -7.97453969e+03 -2.12184356e+03  2.65230445e+02
 -4.71143916e-13  2.35571958e-13  0.00000000e+00]
[ 3.08959761e+03  1.59814572e+03  1.04904038e+19 -1.31130048e+19
 -5.24520190e+19  1.46818466e+03 -3.43035259e+02]
[ 5.99312304e+03  1.69116860e+03  1.12009890e+19 -1.40012362e+19
 -5.60049450e+19  1.56763462e+03 -3.66271331e+02]


# Pályák skálázása kirajzoláshoz

In [105]:
paths = []
paths = results[4:]
scale = 3000
shiftX = 400
shiftY = 550

for i in range(len(paths)):
    for j in range(len(paths[i])):
        if i % 2 == 0:
            paths[i][j] *= scale
            paths[i][j] += shiftX
            paths[i][j] = int(round(paths[i][j]))
        else:
            paths[i][j] *= scale
            paths[i][j] += shiftY
            paths[i][j] = int(round(paths[i][j]))

# GIF készítés

In [106]:
def makegif():
    frames = []
    for file_name in os.listdir(os.getcwd()):
        if file_name.endswith('.jpg'):
            frames.append(file_name)
    with imageio.get_writer('animacio.gif', mode='I', fps=round(periodtime*60)) as writer:
        for filename in frames:
            image = imageio.imread(filename)
            writer.append_data(image)
            os.remove(filename)
        writer.close()

# Animálás

In [107]:
def parseImage(canvas, image, coords):
    height, width = image.shape[:2]
    x = coords[1]
    y = coords[0]
    try:
        canvas[x:x+height, y:y+width,:] = image
    except:
        return

canvas = np.zeros((500,950,3), np.uint8)
x, y = canvas.shape[:2]
gorgosTamasz = cv2.imread('./resources/gorgos_tamasz2.png')
gorgosTamasz = cv2.resize(gorgosTamasz, (50, 50))
csuszka = cv2.imread('./resources/csuszka.png')
csuszka = cv2.resize(csuszka, (75, 50))
blue = (255,127,0)
font = cv2.FONT_HERSHEY_SIMPLEX 
org = (50, 50) 
fontScale = 1
color = (255, 255, 255)
thickness = 2

for i in range(len(paths[0])):

    canvas = np.zeros((500,950,3), np.uint8)
    cv2.putText(canvas, 'Bal sz.h.: 238fok     Jobb sz.h: 35fok ', org, font, fontScale, color, thickness, cv2.LINE_AA) 
    parseImage(canvas, gorgosTamasz, (paths[6][i]-25,y-paths[7][i]))
    parseImage(canvas, gorgosTamasz, (paths[0][i]-25,y-paths[1][i]))
    parseImage(canvas, csuszka, (paths[10][i]-25,y-paths[11][i]-25))
    
    cv2.line(canvas,(paths[0][i],y-paths[1][i]),(paths[2][i],y-paths[3][i]),blue, 5)
    cv2.line(canvas,(paths[2][i],y-paths[3][i]),(paths[4][i],y-paths[5][i]),blue, 5)
    cv2.line(canvas,(paths[4][i],y-paths[5][i]),(paths[6][i],y-paths[7][i]),blue, 5)
    cv2.line(canvas,(paths[8][i],y-paths[9][i]),(paths[10][i],y-paths[11][i]),blue, 5)
    
    cv2.circle(canvas, (paths[0][i],y-paths[1][i]), 12, (100,100,100), -1)
    cv2.circle(canvas, (paths[2][i],y-paths[3][i]), 12, (100,100,100), -1)
    cv2.circle(canvas, (paths[4][i],y-paths[5][i]), 12, (100,100,100), -1)
    cv2.circle(canvas, (paths[6][i],y-paths[7][i]), 12, (100,100,100), -1)
    cv2.circle(canvas, (paths[8][i],y-paths[9][i]), 12, (100,100,100), -1)
    cv2.circle(canvas, (paths[10][i],y-paths[11][i]), 12, (100,100,100), -1)
    cv2.imwrite(('frame%s.jpg' % str(i).zfill(4)), canvas)

makegif()
cv2.imshow('Test', canvas)
cv2.waitKey()
cv2.destroyAllWindows()